### CM1 organization gradient standard output

In [1]:
import sys
import os
import numpy as np
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import metpy.calc as mpc
from metpy.units import units
# for animations
from PIL import Image

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# aggregation indices
os.chdir('/data2/willytsai/python_module')
import conorgidx_revise as agi
from SCAI_calc4obj import labeled_obj, SCAI_calc
from COP_calc4obj import COP
from MICA_calc4obj import MICA

In [4]:
def running_mean(y,window_N):
    y_avg = np.zeros(len(y))
    avg_mask = np.ones(window_N) / window_N

    y_avg = np.convolve(y, avg_mask, 'same')
    y_avg[-window_N:] = y[-window_N:]
    y_avg[:window_N] = y[:window_N]
    return y_avg    

In [5]:
def read_data(data_dir,t_start=0,t_end=721):
    os.chdir(data_dir)
    print(data_dir)
    file_name = glob('*nc')
    data_zon = xr.open_dataset('cm1out_zonmean.nc')
    data_3d = xr.open_dataset('cm1out_sub3d.nc')
    data_2d = xr.open_dataset('cm1out_2Dvars.nc')
    
    # 3d data
    th3d = data_3d.th[t_start:t_end,:30,:,:]
    
    # zonal mean data
    th = data_zon.th[t_start:t_end,:,:,:]
    qv = data_zon.qv[t_start:t_end,:,:,:]
    u = data_zon.uinterp[t_start:t_end,:,:,:]
    v = data_zon.vinterp[t_start:t_end,:,:,:]
    w = data_zon.winterp[t_start:t_end,:,:,:]
    qc = data_zon.qc[t_start:t_end,:,:,:]
    qi = data_zon.qi[t_start:t_end,:,:,:]
    prs = data_zon.prs[t_start:t_end,:,:,:]
    # 2d data
    prate = data_2d.prate[t_start:t_end,:,:]
    pwat = data_2d.pwat[t_start:t_end,:,:]
    cwp = data_2d.cwp[t_start:t_end,:,:]
    cape = data_2d.cape[t_start:t_end,:,:]
    cin = data_2d.cin[t_start:t_end,:,:]
    
    # temperature conversion
    T = th/((1000*100/prs)**(287.15/1004))-273.15 # [degC]
    # density 
    ro = prs/(287.15*(T+273.15))

    x_dim = data_2d.lon
    y_dim = data_2d.lat
    div = np.gradient(v,y_dim*1000,axis=2) # du/dx+dv/dy 
    vor = -np.gradient(u,y_dim*1000,axis=2) # -du/dy+dv/dx
    #relative humidty calculation, Buck (1996)
    es_liq = 0.61121*np.exp((18.678-T/234.5)*(T/(257.14+T)))*10 #[hpa]
    es_ice = 0.61115*np.exp((23.036-T/333.7)*(T/(279.82+T)))*10

    qs_liq = 0.622*es_liq/(prs/100-es_liq)
    qs_ice = 0.622*es_ice/(prs/100-es_ice)

    rh = qv/qs_liq
    rh_ice = qv/qs_ice

    rh = rh.values.flatten()
    rh_ice = rh_ice.values.flatten()
    T_test = T.values.flatten()

    rh[T_test<0] = rh_ice[T_test<0]
    rh = rh.reshape((T.shape[0],T.shape[1],T.shape[2],T.shape[3]))
    rh = xr.DataArray(rh,coords=[T.time,T.lev,T.lat,T.lon],dims=['time','lev','lat','lon'])
    
    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u,v,w,div,vor,cape,cin,prs,ro,th3d)

In [6]:
# get weather or weather2 
server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/'
#server = '/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne'
os.chdir(server)

exp_name = ['CM1_LD2km_RAD4K_MPICTRL_216x216_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U5H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U7H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U12H2km_eqtset2',
            'CM1_LD2km_RAD4K_OPENMPshear_216x216_U15H2km_eqtset2']
print(exp_name)
print('Number of Exp: ', len(exp_name))

['CM1_LD2km_RAD4K_MPICTRL_216x216_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U5H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U7H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U10H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U12H2km_eqtset2', 'CM1_LD2km_RAD4K_OPENMPshear_216x216_U15H2km_eqtset2']
Number of Exp:  6


In [ ]:
exp_label = ['CTRL','U05_H2km','U07_H2km','U10_H2km','U12_H2km','U15_H2km']

In [ ]:
(th1,T1,qv1,qc1,qi1,rh1,prate1,pwat1,cwp1,u1,v1,w1,div1,vor1,cape1,cin1,prs1,ro1,th1_3d) = read_data(server+'/'+exp_name[0],t_start=0)
(th2,T2,qv2,qc2,qi2,rh2,prate2,pwat2,cwp2,u2,v2,w2,div2,vor2,cape2,cin2,prs2,ro2,th2_3d) = read_data(server+'/'+exp_name[1])
(th3,T3,qv3,qc3,qi3,rh3,prate3,pwat3,cwp3,u3,v3,w3,div3,vor3,cape3,cin3,prs3,ro3,th3_3d) = read_data(server+'/'+exp_name[2])
(th4,T4,qv4,qc4,qi4,rh4,prate4,pwat4,cwp4,u4,v4,w4,div4,vor4,cape4,cin4,prs4,ro4,th4_3d) = read_data(server+'/'+exp_name[3])
(th5,T5,qv5,qc5,qi5,rh5,prate5,pwat5,cwp5,u5,v5,w5,div5,vor5,cape5,cin5,prs5,ro5,th5_3d) = read_data(server+'/'+exp_name[4])
(th6,T6,qv6,qc6,qi6,rh6,prate6,pwat6,cwp6,u6,v6,w6,div6,vor6,cape6,cin6,prs6,ro6,th6_3d) = read_data(server+'/'+exp_name[5])
#(th7,T7,qv7,qc7,qi7,rh7,prate7,pwat7,cwp7,u7,v7,w7,div7,vor7,cape7,cin7,prs7,ro7,th7_3d) = read_data(server+'/'+exp_name[6])

# (th5,T5,qv5,qc5,qi5,prate5,pwat5,cwp5,u5,v5,w5,div5,vor5,cape5,cin5) = read_data(server+'/'+exp_name[4])

In [ ]:
# read brightness data
# data2 = xr.open_dataset(server+'/'+exp_name[0]+'/cm1out_BT.nc')
# BT1 = data2.T # brightness temperature determined by cloud top temperature
# data2 = xr.open_dataset(server+'/'+exp_name[1]+'/cm1out_BT.nc')
# BT2 = data2.T # brightness temperature determined by cloud top temperature
# data2 = xr.open_dataset(server+'/'+exp_name[2]+'/cm1out_BT.nc')
# BT3 = data2.T # brightness temperature determined by cloud top temperature
# data2 = xr.open_dataset(server+'/'+exp_name[3]+'/cm1out_BT.nc')
# BT4 = data2.T # brightness temperature determined by cloud top temperature
# data2 = xr.open_dataset(server+'/'+exp_name[4]+'/cm1out_BT.nc')
# BT5 = data2.T # brightness temperature determined by cloud top temperature
# data2 = xr.open_dataset(server+'/'+exp_name[5]+'/cm1out_BT.nc')
# BT6 = data2.T # brightness temperature determined by cloud top temperature
#data2 = xr.open_dataset(server+'/'+exp_name[6]+'/cm1out_BT.nc')
#BT7 = data2.T # brightness temperature determined by cloud top temperature

In [ ]:
x_dim = pwat1.lon
y_dim = pwat1.lat
z_dim = T1.lev
t_dim = np.arange(len(T1.time))/3

In [ ]:
fig_out = '/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/fig_analysis/'

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(9,6))

QR = np.ones(len(z_dim))
for n in range(65):
    if z_dim[n]<=12:
        QR[n] = -4
    elif z_dim[n]>12 and z_dim[n] <= 15:
        QR[n] = -4*(15-z_dim[n])/3.
    else:
        QR[n] = 0
        
ax[0].plot(QR,z_dim,'-ok')
ax[0].set_xlabel('[K/d]',fontsize=14)
ax[0].set_ylabel('[km]',fontsize=14)
ax[0].grid(linestyle=':')
ax[0].set_title('Radiative cooling [K/d]')

ax[1].plot(ro1[0,:,:,:].mean(axis=(1,2))*1004*QR,z_dim,'-ok')
ax[1].set_xlabel('[J/m^3/d]',fontsize=14)
ax[1].set_ylabel('[km]',fontsize=14)
ax[1].grid(linestyle=':')
ax[1].set_title('Radiation [J/m^3/d], Q$_R$')

vint_QR = np.trapz(ro1[0,:,:,:].mean(axis=(1,2))*1004*QR,z_dim*1000)/86400
print(np.trapz(ro1[0,:,:,:].mean(axis=(1,2))*1004*QR,z_dim*1000)/86400)
#fig.savefig(fig_out+'radiative_cooling.png',dpi=200,bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(9,6))

QR = np.ones(len(z_dim))
for n in range(65):
    if z_dim[n]>5 and z_dim[n]<17:
        QR[n] = -4
    elif z_dim[n]>2 and z_dim[n] <= 5:
        QR[n] = -4*(z_dim[n]-2)/3.
    else:
        QR[n] = 0

vint_QR2 = np.trapz(ro1[0,:,:,:].mean(axis=(1,2))*1004*QR,z_dim*1000)/86400
factor = vint_QR/vint_QR2

ax[0].plot(QR*factor,z_dim,'-ok')
ax[0].set_xlabel('[K/d]',fontsize=14)
ax[0].set_ylabel('[km]',fontsize=14)
ax[0].grid(linestyle=':')
ax[0].set_title('Radiative cooling [K/d]')

ax[1].plot(ro1[0,:,:,:].mean(axis=(1,2))*1004*QR*factor,z_dim,'-ok')
ax[1].set_xlabel('[J/m^3/d]',fontsize=14)
ax[1].set_ylabel('[km]',fontsize=14)
ax[1].grid(linestyle=':')
ax[1].set_title('Radiation [J/m^3/d], Q$_R$')

print(vint_QR2)
# #fig.savefig(fig_out+'radiative_cooling.png',dpi=200,bbox_inches='tight')

In [ ]:
a = np.empty(721)
for t in range(721):
    a[t] = np.trapz(ro6[t,:,:,:].mean(axis=(1,2))*1004*QR,z_dim*1000)/86400
plt.plot(a)
plt.ylim([-398,-395.5])

In [ ]:
fig = plt.figure(figsize=(8,6))

plt.contour(x_dim,y_dim,prate1[-360,:,:],levels=[10,20],colors=['k']
           ,linewidths=1)
plt.pcolor(x_dim,y_dim,pwat1[-360,:,:],cmap='Spectral',
          vmax=65,vmin=45)
cbar = plt.colorbar(shrink=0.8)
cbar.set_label('[mm]',fontsize=12)
plt.title(exp_name[0]+',  Time= '+ str(np.round(360/3,2)) + ' (h)',fontsize=13)

In [ ]:
# hovmuller diagram for PWAT and P_y cross
fig,ax = plt.subplots(len(exp_name),1,figsize=(10,6))

for n,(pwat,prate) in enumerate(zip([pwat1,pwat2]
                                    ,[prate1,prate2])):
    cf = ax[n].pcolor(t_dim,y_dim,pwat.mean('lon').T,cmap='Spectral',vmin=55,vmax=65)
    cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
               bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
               borderpad=0) 
    cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical')
    cbar.set_label('PW [mm]')

    ax[n].contour(t_dim,y_dim,prate.mean('lon').T,levels=[20],colors=['k',]
                 ,linewidths=1)   
    ax[n].set_title(exp_label[n],fontsize=14,loc='right')
    ax[n].set_ylabel('[km]',fontsize=13)
    if n != len(exp_label)-1: 
        ax[n].set_xticks([])
        
    #ax[n].set_xlim([200,220])

ax[n].set_xlabel('Time [hr]',fontsize=13)
plt.tight_layout()
#fig.savefig(fig_out+'PWAT_PREC_hovmuller.png',dpi=200,bbox_inches='tight')

### Snapshot of evolutions

In [ ]:
fig,ax = plt.subplots(1,1)
cf = ax.pcolormesh(prate2[500,:,:]/24,vmin=1,cmap='jet')
cf.set_clim(1,60)
#plt.colorbar(cf)
cf.cmap.set_under('w')

In [ ]:
fig,ax = plt.subplots(4,4,figsize=(10,10))

t=513

cf = ax[0,0].pcolor(prate1[t,:,:]/24,vmin=1,cmap='jet');ax[0,0].set_xticks([]);ax[0,0].set_yticks([]);cf.set_clim(1,60)
cf = ax[0,1].pcolor(prate1[t+6,:,:]/24,vmin=1,cmap='jet');ax[0,1].set_xticks([]);ax[0,1].set_yticks([]);cf.set_clim(1,60)
cf = ax[0,2].pcolor(prate1[t+12,:,:]/24,vmin=1,cmap='jet');ax[0,2].set_xticks([]);ax[0,2].set_yticks([]);cf.set_clim(1,60)
cf = ax[0,3].pcolor(prate1[t+18,:,:]/24,vmin=1,cmap='jet');ax[0,3].set_xticks([]);ax[0,3].set_yticks([]);cf.set_clim(1,60)

cf = ax[1,0].pcolor(prate2[t,:,:]/24,vmin=1,cmap='jet');ax[1,0].set_xticks([]);ax[1,0].set_yticks([]);cf.set_clim(1,60)
cf = ax[1,1].pcolor(prate2[t+6,:,:]/24,vmin=1,cmap='jet');ax[1,1].set_xticks([]);ax[1,1].set_yticks([]);cf.set_clim(1,60)
cf = ax[1,2].pcolor(prate2[t+12,:,:]/24,vmin=1,cmap='jet');ax[1,2].set_xticks([]);ax[1,2].set_yticks([]);cf.set_clim(1,60)
cf = ax[1,3].pcolor(prate2[t+18,:,:]/24,vmin=1,cmap='jet');ax[1,3].set_xticks([]);ax[1,3].set_yticks([]);cf.set_clim(1,60)

cf = ax[2,0].pcolor(prate4[t,:,:]/24,vmin=1,cmap='jet');ax[2,0].set_xticks([]);ax[2,0].set_yticks([]);cf.set_clim(1,60)
cf = ax[2,1].pcolor(prate4[t+6,:,:]/24,vmin=1,cmap='jet');ax[2,1].set_xticks([]);ax[2,1].set_yticks([]);cf.set_clim(1,60)
cf = ax[2,2].pcolor(prate4[t+12,:,:]/24,vmin=1,cmap='jet');ax[2,2].set_xticks([]);ax[2,2].set_yticks([]);cf.set_clim(1,60)
cf = ax[2,3].pcolor(prate4[t+18,:,:]/24,vmin=1,cmap='jet');ax[2,3].set_xticks([]);ax[2,3].set_yticks([]);cf.set_clim(1,60)

cf = ax[3,0].pcolor(prate6[t,:,:]/24,vmin=1,cmap='jet');ax[3,0].set_xticks([]);ax[3,0].set_yticks([]);cf.set_clim(1,60)
cf = ax[3,1].pcolor(prate6[t+6,:,:]/24,vmin=1,cmap='jet');ax[3,1].set_xticks([]);ax[3,1].set_yticks([]);cf.set_clim(1,60)
cf = ax[3,2].pcolor(prate6[t+12,:,:]/24,vmin=1,cmap='jet');ax[3,2].set_xticks([]);ax[3,2].set_yticks([]);cf.set_clim(1,60)
cf = ax[3,3].pcolor(prate6[t+18,:,:]/24,vmin=1,cmap='jet');ax[3,3].set_xticks([]);ax[3,3].set_yticks([]);cf.set_clim(1,60)

ax[0,0].set_title('t= '+str(t/3)+' (h)',loc='right',fontsize=11)
ax[0,1].set_title('t= '+str((t+6)/3)+' (h)',loc='right',fontsize=11)
ax[0,2].set_title('t= '+str((t+12)/3)+' (h)',loc='right',fontsize=11)
ax[0,3].set_title('t= '+str((t+18)/3)+' (h)',loc='right',fontsize=11)

plt.tight_layout()

In [ ]:
#fig.savefig(fig_out+'PRECsnapshots_4exps.png',dpi=200,bbox_inches='tight')

In [ ]:
# time
#color_label=['k','darkred','r','darkgreen','limegreen','orange','gold']
color_label2=['k','gold','orange','limegreen','darkgreen','r','darkred']

#color_label=['k','darkred']

fig,ax = plt.subplots(2,1,figsize=(10,7))
for n,(pwat,prate) in enumerate(zip([pwat1,pwat2,pwat3,pwat4,pwat5,pwat6]
                                    ,[prate1,prate2,prate3,prate4,prate5,prate6])):

    ax[0].plot(pwat.mean(('lon','lat')),color=color_label2[n],linewidth=2)
    ax[1].plot(prate.mean(('lon','lat')),color=color_label2[n],linewidth=2)

ax[0].set_ylabel('CWV [mm]',fontsize=14)
ax[0].set_xlim([0,241]);ax[0].set_ylim([56,66])
ax[1].set_ylabel('Prec [mm/d]',fontsize=14)
ax[1].set_xlim([0,241]);ax[1].set_ylim([5,20])
ax[0].set_xticks(np.linspace(0,720,11));ax[0].set_xticklabels(np.linspace(0,10,11))
ax[1].set_xticks(np.linspace(0,720,11));ax[1].set_xticklabels(np.linspace(0,10,11))

ax[0].legend(exp_label,frameon=False,ncol=2)
ax[0].grid(linestyle=':')
ax[1].grid(linestyle=':')
ax[1].set_xlabel('Simulation time [day]',fontsize=13)

ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)

#plt.rcParams(labelsize=13)
#fig.savefig(fig_out+'domain_averaged_PW_PREC.png',dpi=200,bbox_inches='tight')

In [ ]:
pm1 = prate1[-360:,:,:].mean(('lat','lon'))
pm2 = prate2[-360:,:,:].mean(('lat','lon'))
pm3 = prate3[-360:,:,:].mean(('lat','lon'))
pm4 = prate4[-360:,:,:].mean(('lat','lon'))
pm5 = prate5[-360:,:,:].mean(('lat','lon'))
pm6 = prate6[-360:,:,:].mean(('lat','lon'))

pstd1 = pm1.std()
pstd2 = pm2.std()
pstd3 = pm3.std()
pstd4 = pm4.std()
pstd5 = pm5.std()
pstd6 = pm6.std()

In [ ]:
exp_label

In [ ]:
fig = plt.figure(figsize=(5.5,6.5))

for n,(pm,pstd) in enumerate(zip([pm1,pm2,pm3,pm4,pm5,pm6],[pstd1,pstd2,pstd3,pstd4,pstd5,pstd6])):
    plt.errorbar(n,pm.mean(),pstd,marker='s',ms=12,color=color_label2[n])
plt.xticks(range(len(exp_label)),exp_label)
plt.ylabel('Domain-averaged P [mm/d]',fontsize=14)
plt.grid(linestyle=':')
plt.xlim([-0.5,5.5]);plt.ylim([10.5,13.5])
plt.tick_params(labelsize=12)
#fig.savefig(fig_out+'errorbar_DMPREC.png',dpi=200,bbox_inches='tight')

In [ ]:
Z = np.tile(z_dim*1000,reps=(len(t_dim),len(y_dim),1)).swapaxes(1,2)

In [ ]:
# MSE time-series
vint_mse = np.zeros((len(exp_name),len(t_dim),216))
vint_Lqv = np.copy(vint_mse)
vint_CpT = np.copy(vint_mse)

for n,(T,qv) in enumerate(zip([T1,T2,T3,T4,T5,T6],[qv1,qv2,qv3,qv4,qv5,qv6])):
    vint_mse[n,:,:] = np.trapz(1004*(T.squeeze()+273.15)+9.8*Z*1000+2.5e6*qv.squeeze(),z_dim*1000,axis=1).squeeze()
    vint_Lqv[n,:,:] = np.trapz(2.5e6*qv,z_dim*1000,axis=1).squeeze()
    vint_CpT[n,:,:] = np.trapz(1004*(T+273.15),z_dim*1000,axis=1).squeeze()

In [ ]:
fig = plt.figure(figsize=(10,5))

for n in range(len(exp_name)):
    plt.plot(t_dim,vint_mse[n,:,:].mean(axis=1)/1000,color=color_label[n],label=exp_name[n],
            linewidth=1.8)
plt.legend(frameon=False)
    
# quantify the drifting of CMSE
del_mse_1deg = np.trapz(1004*np.ones(len(z_dim)),z_dim*1000,axis=0).squeeze()/1000    
plt.plot(t_dim,vint_mse[0,0,:].mean(axis=0)*np.ones(len(t_dim))/1000)

factor=0.5
plt.fill_between(t_dim,vint_mse[0,0,:].mean(axis=0)/1000+factor*del_mse_1deg*np.ones(len(t_dim))
                 ,vint_mse[0,0,:].mean(axis=0)/1000-factor*del_mse_1deg*np.ones(len(t_dim)),color='lightgrey',alpha=0.4)
    
#plt.ylim([23000,23200])
plt.xlim([0,240])
plt.ylabel('CMSE [KJ/kg*m]')
plt.xlabel('Time [hr]')
plt.grid(linestyle=':')

In [ ]:
# decompose into <T> & <qv> contributions
fig = plt.figure(figsize=(10,5))

for n in range(len(exp_name)):
    plt.plot(t_dim,vint_CpT[n,:,:].mean(axis=1)/1000-vint_CpT[n,0,:].mean()/1000,color=color_label[n],label=exp_name[n],
            linewidth=1.8)
plt.legend(frameon=False)
    
plt.xlim([0,240])
plt.ylabel('Vint_CpT [KJ/kg*m]')
plt.xlabel('Time [hr]')
plt.grid(linestyle=':')

In [ ]:
# decompose into <T> & <qv> contributions
fig = plt.figure(figsize=(10,5))

for n in range(len(exp_name)):
    plt.plot(t_dim,vint_Lqv[n,:,:].mean(axis=1)/1000-vint_Lqv[n,0,:].mean()/1000,color=color_label[n],label=exp_name[n],
            linewidth=1.8)
plt.legend(frameon=False)
    
plt.xlim([0,240])
plt.ylabel('Vint_Lqv [KJ/kg*m]')
plt.xlabel('Time [hr]')
plt.grid(linestyle=':')

In [ ]:
# domain-averaged z-t plane
fig,ax = plt.subplots(len(exp_name),1,figsize=(8,15))

for n,data in enumerate([rh1,rh2,rh3,rh4,rh5,rh6]):
    ax[n].contourf(t_dim,z_dim,data.mean(axis=(2,3)).T,levels=20,cmap='jet_r')
    ax[n].set_title(exp_name[n])
    ax[n].set_ylabel('Height [km]')
    ax[n].set_ylim([0,18])
plt.xlabel('Time [hr]')
plt.tight_layout()

In [ ]:
# plotting zonal mean
# y cross-section
marker_label=['s','o']

fig,ax = plt.subplots(2,1,figsize=(9,7))

for n,data in enumerate([pwat1,pwat2,pwat3,pwat4,pwat5,pwat6]):
    ax[0].plot(x_dim,np.nanmean(data[-360:,:,:],axis=(0,1)),color=color_label2[n]
              ,label=exp_label[n],linewidth=2)
    #ax[0].plot(x_dim,np.ones(len(x_dim))*np.nanmean(data[-360:,:,:]),'--',color=color_label[n])
    
    ax[1].plot(y_dim,np.nanmean(data[-360:,:,:],axis=(0,2)),color=color_label2[n]
              ,label=exp_label[n],linewidth=2)
#    ax[1].plot(y_dim,np.ones(len(y_dim))*np.nanmean(data[-360:,:,:]),'--',color=color_label[n])

ax[0].legend(frameon=False,fontsize=8,ncol=2);
ax[0].set_xlabel('x direction [km]',fontsize=14);ax[0].set_ylim([57,63])
ax[0].set_ylabel('CWV [mm]',fontsize=14)
ax[1].set_xlabel('y direction [km]',fontsize=14);ax[1].set_ylim([57,63])
ax[1].set_ylabel('CWV [mm]',fontsize=14)
ax[0].set_xlim([0,432])
ax[1].set_xlim([0,432])
plt.tight_layout(h_pad=1)
ax[0].grid(linestyle=':')
ax[1].grid(linestyle=':')
ax[1].legend(frameon=False,fontsize=8,ncol=2);
ax[0].tick_params(labelsize=12)
ax[1].tick_params(labelsize=12)

ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)

fig.savefig(fig_out+'PWAT_crosssection.png',dpi=200,bbox_inches='tight')

In [ ]:
# y cross-section 
marker_label=['s','o']

fig,ax = plt.subplots(2,1,figsize=(9,7))

for n,data in enumerate([prate1,prate2,prate3,prate4,prate5,prate6]):
    ax[0].plot(x_dim,running_mean(np.nanmean(data[-360:,:,:],axis=(0,1)),window_N=3),color=color_label2[n]
              ,label=exp_label[n],linewidth=2)
    
    ax[1].plot(y_dim,running_mean(np.nanmean(data[-360:,:,:],axis=(0,2)),window_N=3),color=color_label2[n]
              ,label=exp_label[n],linewidth=2)
    ax[0].plot(x_dim,np.ones(len(x_dim))*np.nanmean(data[-360:,:,:]),'--',color=color_label[n])
    ax[1].plot(y_dim,np.ones(len(y_dim))*np.nanmean(data[-360:,:,:]),'--',color=color_label[n])

ax[0].legend(frameon=False,fontsize=8,ncol=2);
ax[1].legend(frameon=False,fontsize=8,ncol=2);

ax[0].set_xlabel('x direction [km]',fontsize=14);ax[0].set_xlim([0,432])
ax[0].set_ylim([6,22])
ax[0].set_ylabel('Precip [mm/d]',fontsize=14)
ax[1].set_xlabel('y direction [km]',fontsize=14);ax[1].set_xlim([0,432])
ax[1].set_ylabel('Precip [mm/d]',fontsize=14)
ax[1].set_ylim([6,22])
ax[0].grid(linestyle=':')
ax[1].grid(linestyle=':')

plt.tight_layout(h_pad=0.5)
ax[0].tick_params(labelsize=12)
ax[1].tick_params(labelsize=12)

ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)

fig.savefig(fig_out+'PREC_crosssection.png',dpi=200,bbox_inches='tight')

In [ ]:
%%time
# streamfunction, anelastic + boussinesq==> grad(ro*V) = 0
strmf1 = np.zeros((len(t_dim),len(z_dim),len(y_dim)))
strmf2 = np.zeros((len(t_dim),len(z_dim),len(y_dim)))
strmf3 = np.zeros((len(t_dim),len(z_dim),len(y_dim)))
strmf4 = np.zeros((len(t_dim),len(z_dim),len(y_dim)))
strmf5 = np.zeros((len(t_dim),len(z_dim),len(y_dim)))
strmf6 = np.zeros((len(t_dim),len(z_dim),len(y_dim)))

for t in range(len(t_dim)):
    for n in range(len(z_dim)):
        strmf1[t,n,:] = np.trapz(ro1[t,:n,:,0]*v1[t,:n,:,0],z_dim[:n]*1000,axis=0) 
        strmf2[t,n,:] = np.trapz(ro2[t,:n,:,0]*v2[t,:n,:,0],z_dim[:n]*1000,axis=0)
        strmf3[t,n,:] = np.trapz(ro3[t,:n,:,0]*v3[t,:n,:,0],z_dim[:n]*1000,axis=0)
        strmf4[t,n,:] = np.trapz(ro4[t,:n,:,0]*v4[t,:n,:,0],z_dim[:n]*1000,axis=0)
        strmf5[t,n,:] = np.trapz(ro5[t,:n,:,0]*v5[t,:n,:,0],z_dim[:n]*1000,axis=0)
        strmf6[t,n,:] = np.trapz(ro6[t,:n,:,0]*v6[t,:n,:,0],z_dim[:n]*1000,axis=0)

In [ ]:
# RH cross section
fig,ax = plt.subplots(len(exp_name),1,figsize=(8,10))
 
for n,rh in enumerate([rh1,rh2,rh3,rh4,rh5,rh6]):
    cf=ax[n].contourf(y_dim,z_dim,rh[-360:,:,:,0].mean(axis=0),levels=np.linspace(0.4,1,13),linewidths=2,cmap='jet_r')
    ax[n].contour(y_dim,z_dim,rh[-360:,:,:,0].mean(axis=0),levels=[0.5,0.6],linewidths=1.5,colors=['k'])
    ax[n].set_ylim([0,18])
    if n == 0:
        cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
                   bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
                   borderpad=0) 
        cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical')

    ax[n].set_ylabel('height [km]',fontsize=13)
    ax[n].grid(linestyle=':')
    ax[n].set_title(exp_name[n],fontsize=14)
    
ax[n].set_xlabel('y direction [km]',fontsize=13)
plt.tight_layout()

In [ ]:
# T cross section
fig,ax = plt.subplots(len(exp_name)-1,1,figsize=(8,10))
 
for n,T in enumerate([T2,T3,T4,T5,T6]):
    cf=ax[n].contourf(y_dim,z_dim,T[-360:,:,:,0].mean(axis=0)-T1[-360:,:,:,0].mean(axis=0),levels=20,linewidths=2,cmap='RdBu_r')
    #ax[n].contour(y_dim,z_dim,T[-360:,:,:,0].mean(axis=0),levels=['40'],linewidths=2,colors=['grey'])
    ax[n].set_ylim([0,18])
    if n == 0:
        cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
                   bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
                   borderpad=0) 
        cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical',shrink=1)

    ax[n].set_ylabel('height [km]',fontsize=13)
    ax[n].grid(linestyle=':')
    ax[n].set_title(exp_label[n+1],fontsize=14)
    
ax[n].set_xlabel('y direction [km]',fontsize=13)
plt.tight_layout()

In [ ]:
# T cross section
fig,ax = plt.subplots(len(exp_name)-1,1,figsize=(8,10))

for n,qv in enumerate([qv2,qv3,qv4,qv5,qv6]):
    cf=ax[n].contourf(y_dim,z_dim,1000*qv[-360:,:,:,0].mean(axis=0)-1000*qv1[-360:,:,:,0].mean(axis=0),levels=np.linspace(-1.5,1.5,21),linewidths=2,cmap='RdBu')
    ax[n].contour(y_dim,z_dim,1000*qv[-360:,:,:,0].mean(axis=0)-1000*qv1[-360:,:,:,0].mean(axis=0),levels=[0],colors='k')
    #ax[n].contour(y_dim,z_dim,T[-360:,:,:,0].mean(axis=0),levels=['40'],linewidths=2,colors=['grey'])
    ax[n].set_ylim([0,18])
    if n == 0:
        cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
                   bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
                   borderpad=0) 
        cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical',shrink=1)

    ax[n].set_ylabel('height [km]',fontsize=13)
    ax[n].grid(linestyle=':')
    ax[n].set_title(exp_name[n+1],fontsize=14)
    
ax[n].set_xlabel('y direction [km]',fontsize=13)
plt.tight_layout()

In [ ]:
# U cross section
fig,ax = plt.subplots(len(exp_name)-1,1,figsize=(8,10))

for n,u in enumerate([u2,u3,u4,u5,u6]):
    cf=ax[n].contourf(y_dim,z_dim,u[0,:,:,0],levels=np.linspace(1,15,8),linewidths=2,cmap='RdBu')
    #ax[n].contour(y_dim,z_dim,v[-360:,:,:,0].mean(axis=0),levels=[0],colors='k')
    #ax[n].contour(y_dim,z_dim,T[-360:,:,:,0].mean(axis=0),levels=['40'],linewidths=2,colors=['grey'])
    #ax[n].set_ylim([0,18])
    if n == 4:
        cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
                   bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[n].transAxes,
                   borderpad=0) 
        cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical',shrink=1)
        cbar.set_label('U [m/s]')

    ax[n].set_ylabel('height [km]',fontsize=13)
    ax[n].grid(linestyle=':')
    ax[n].set_title(exp_label[n+1],fontsize=14,loc='right')
    ax[n].set_ylim([0,6])    
    ax[n].set_xlabel('y direction [km]',fontsize=13)
plt.tight_layout()
fig.savefig(fig_out+'Ushear_crossection.png',dip=200,bbox_inches='tight')

In [ ]:
# V cross section
fig,ax = plt.subplots(len(exp_name)-1,1,figsize=(8,10))

for n,v in enumerate([v2,v3,v4,v5,v6]):
    cf=ax[n].contourf(y_dim,z_dim,v[-360:,:,:,0].mean(axis=0),levels=np.linspace(-1.5,1.5,21),linewidths=2,cmap='RdBu')
    ax[n].contour(y_dim,z_dim,v[-360:,:,:,0].mean(axis=0),levels=[0],colors='k')
    #ax[n].contour(y_dim,z_dim,T[-360:,:,:,0].mean(axis=0),levels=['40'],linewidths=2,colors=['grey'])
    ax[n].set_ylim([0,18])
    if n == 0:
        cbaxes = inset_axes(ax[n], width="2%",height="70%",loc='lower left',
                   bbox_to_anchor=(1.02,0.15, 1, 1),bbox_transform=ax[0].transAxes,
                   borderpad=0) 
        cbar=fig.colorbar(cf,cax=cbaxes,orientation='vertical',shrink=1)

    ax[n].set_ylabel('height [km]',fontsize=13)
    ax[n].grid(linestyle=':')
    ax[n].set_title(exp_name[n+1],fontsize=14)
    
ax[n].set_xlabel('y direction [km]',fontsize=13)
plt.tight_layout()

In [ ]:
# soundings over 6 places
y_pos = [26,52,78,104,130,156,182]
color_snds = ['green','r','purple','purple','purple','r','green']
ncol = [0,1,2,3,0,1,2,3]
nrow = [0,0,0,0,1,1,1,1]
pos_label = ['NS','EG1','S','S','S','EG2','NS']

fig,ax = plt.subplots(2,4,figsize=(20,10))

for n,(T,qv,prs) in enumerate(zip([T1,T2,T3,T4,T5,T6],[qv1,qv2,qv3,qv4,qv5,qv6],
                                 [prs1,prs2,prs3,prs4,prs5,prs6])):
    
    mse = 1004*(T[-360:,:,:,0]+273.15).mean(axis=0)+2.5e6*qv[-360:,:,:,0].mean(axis=0) \
            +9.8*np.tile(z_dim*1000,reps=(len(y_dim),1)).swapaxes(0,1)
    dse = 1004*(T[-360:,:,:,0]+273.15).mean(axis=0) \
            +9.8*np.tile(z_dim*1000,reps=(len(y_dim),1)).swapaxes(0,1)
        
    for p,y in enumerate(y_pos):
    
        # smse
        qvs = mpc.saturation_mixing_ratio(prs[-360:,:,y,0].mean(axis=(0)).values/100*units('mbar')
                            ,T[-360:,:,y,0].mean(axis=(0)).values*units('degC'))
        smse = 1004*(T[-360:,:,y,0]+273.15).mean(axis=0)+2.5e6*qvs \
                +9.8*z_dim*1000
        
        ax[nrow[n],ncol[n]].plot(dse[:,y]/1000,z_dim,color=color_snds[p],linestyle='--')
        ax[nrow[n],ncol[n]].plot(mse[:,y]/1000,z_dim,color=color_snds[p],linestyle='-',label=pos_label[p])
        ax[nrow[n],ncol[n]].plot(smse/1000,z_dim,color=color_snds[p],linestyle='-.')

        ax[nrow[n],ncol[n]].set_title(exp_label[n])
        ax[nrow[n],ncol[n]].set_ylim([0,18])
        ax[nrow[n],ncol[n]].set_xlim([300,360])
        ax[nrow[n],ncol[n]].grid(linestyle=':')
ax[0,0].legend()

In [ ]:
# soundings over 6 places, cape and cin values
y_pos = [26,52,78,104,130,156,182]
color_snds = ['green','r','purple','purple','purple','r','green']
#nrow = [0,0,1,1,2,2,3]
#ncol = [0,1,0,1,0,1,0]
pos_label = ['NS','EG1','S','S','S','EG2','NS']

fig,ax = plt.subplots(1,2,figsize=(10,5))

for n,(cape,cin) in enumerate(zip([cape1,cape2,cape3,cape4,cape5,cape6]
                                  ,[cin1,cin2,cin3,cin4,cin5,cin6])):    
    for p,y in enumerate(y_pos):
        ax[0].plot(p,cape[-360:,y,:].mean(),'^',color=color_label[n])
        ax[1].plot(p,cin[-360:,y,:].mean(),'o',color=color_label[n])

ax[0].set_xticks(np.arange(len(y_pos)))
ax[1].set_xticks(np.arange(len(y_pos)))
ax[0].set_xticklabels(pos_label)
ax[1].set_xticklabels(pos_label)

In [ ]:
%%time
fig,ax = plt.subplots(2,3,figsize=(12,8))
nrow = [0,0,0,1,1,1]
ncol = [0,1,2,0,1,2]
clevs = np.linspace(-1000,1000,21)

for n,(strmf,rh) in enumerate(zip([strmf1,strmf2,strmf3,strmf4,strmf5,strmf6]
                         ,[rh1,rh2,rh3,rh4,rh5,rh6])):
    ax[nrow[n],ncol[n]].contour(y_dim,z_dim,strmf[-360:,:,:].mean(axis=0),levels=clevs,linewidths=1,colors='k')
    ax[nrow[n],ncol[n]].contour(y_dim,z_dim,strmf[-360:,:,:].mean(axis=0),levels=['0'],linewidths=2,colors=['w'])
    cf = ax[nrow[n],ncol[n]].pcolor(y_dim,z_dim,100*rh[-360:,:,:,0].mean(axis=0),vmin=40,vmax=100,cmap='BrBG')
    #cbar = plt.colorbar(cf,ax=ax[nrow[n],ncol[n]])
    #cbar.set_label('RH [%]',fontsize=12)
    ax[nrow[n],ncol[n]].set_ylim([0,18])

    ax[nrow[n],ncol[n]].set_ylabel('height [km]',fontsize=13)
    ax[nrow[n],ncol[n]].grid(linestyle=':')
    ax[nrow[n],ncol[n]].set_title(exp_label[n],fontsize=14,loc='right')
    if nrow[n] != 1:
        ax[nrow[n],ncol[n]].set_xticklabels([])
    if nrow[n] == 1:
        ax[nrow[n],ncol[n]].set_xlabel('y direction [km]',fontsize=13)
    
    if n == len(exp_label)-1:
        cbaxes = fig.add_axes([1.01, 0.2, 0.02, 0.6]) 
        cb = plt.colorbar(cf, cax = cbaxes)  
        cb.set_label('Relative humidity [%]',fontsize=13)
    
plt.tight_layout()
fig.savefig(fig_out+'rh_strm_crosssection.png',dip=200,bbox_inches='tight')

In [ ]:
%%time
fig,ax = plt.subplots(2,3,figsize=(12,8))
nrow = [0,0,0,1,1,1]
ncol = [0,1,2,0,1,2]
clevs = np.linspace(-0.2,0.2,11)

for n,(strmf,rh) in enumerate(zip([ro1*w1,ro1*w2,ro1*w3,ro1*w4,ro1*w5,ro1*w6]
                         ,[rh1,rh2,rh3,rh4,rh5,rh6])):
    cf=ax[nrow[n],ncol[n]].contourf(y_dim,z_dim,-9.8*strmf[-360:,:,:,0].mean(axis=0),levels=clevs,linewidths=1,cmap='RdBu')
    #ax[nrow[n],ncol[n]].contour(y_dim,z_dim,9.8*strmf[-360:,:,:,0].mean(axis=0),levels=clevs,linewidths=0.5,color='k')
    #ax[nrow[n],ncol[n]].contour(y_dim,z_dim,strmf[-360:,:,:,0].mean(axis=0),levels=['0'],linewidths=2,colors=['w'])
    #cf = ax[nrow[n],ncol[n]].pcolor(y_dim,z_dim,100*rh[-360:,:,:,0].mean(axis=0),vmin=40,vmax=100,cmap='BrBG')
    #cbar = plt.colorbar(cf,ax=ax[nrow[n],ncol[n]])
    #cbar.set_label('RH [%]',fontsize=12)
    ax[nrow[n],ncol[n]].set_ylim([0,18])

    ax[nrow[n],ncol[n]].set_ylabel('height [km]',fontsize=13)
    ax[nrow[n],ncol[n]].grid(linestyle=':')
    ax[nrow[n],ncol[n]].set_title(exp_label[n],fontsize=14,loc='right')
    if nrow[n] != 1:
        ax[nrow[n],ncol[n]].set_xticklabels([])
    if nrow[n] == 1:
        ax[nrow[n],ncol[n]].set_xlabel('y direction [km]',fontsize=13)
    
    if n == len(exp_label)-1:
        cbaxes = fig.add_axes([1.01, 0.2, 0.02, 0.6]) 
        cb = plt.colorbar(cf, cax = cbaxes)  
        cb.set_label('Omega [Pa/s]',fontsize=13)
    
plt.tight_layout()
#fig.savefig(fig_out+'rh_strm_crosssection.png',dip=200,bbox_inches='tight')

In [ ]:
# cloud fraction 
fig,ax = plt.subplots(len(exp_label),1,figsize=(8,14))
 
for n,(qc,qi) in enumerate(zip([qc1,qc2,qc3,qc4,qc5,qc6]
                         ,[qi1,qi2,qi3,qi4,qi5,qi6])):
    #ax[n].contour(y_dim,z_dim,strmf[-360:,:,:].mean(axis=0),levels=10,linewidths=1,colors='k')
    #ax[n].contour(y_dim,z_dim,strmf[-360:,:,:].mean(axis=0),levels=['0'],linewidths=2,colors=['grey'])
    cf = ax[n].pcolor(y_dim,z_dim,qc[-360:,:,:,0].mean(axis=0)+qi[-360:,:,:,0].mean(axis=0),vmin=1e-5,cmap='BrBG')
    cbar = plt.colorbar(cf,ax=ax[n])
    ax[n].set_ylim([0,18])

    ax[n].set_ylabel('height [km]',fontsize=13)
    ax[n].grid(linestyle=':')
    ax[n].set_title(exp_name[n],fontsize=14)
    
ax[n].set_xlabel('y direction [km]',fontsize=13)
plt.tight_layout()

In [ ]:
# lumpsize distribution 

def lumpsize_dist(prec):
    
    size_cri = np.linspace(0,400,101)
    lumpsize_dist = np.zeros((2,len(size_cri)-1))

    for t in range(prec.shape[0]):
        pr_s = prec[t,65:151,:] # only in the shear region
        pr_ns = np.concatenate((prec[t,-43:,:],prec[t,:43,:]))
        cldmask_s = np.zeros(pr_s.shape)
        cldmask_ns = np.copy(cldmask_s)
        cldmask_s[pr_s> 5] = 1 # cloudmaks, prec>5mm/d, cold cloud
        cldmask_ns[pr_ns> 5] = 1 # cloudmaks, prec>5mm/d, cold cloud
        
        try:
        # shear region
            labeled_array, num_feature = labeled_obj(cldmask_s,1,1)
            for label in np.unique(labeled_array)[1:]:
                tmp = len(np.where(labeled_array == label)[0])
                for i in range(len(size_cri)-1):
                    if (tmp >= size_cri[i]) and (tmp < size_cri[i+1]):
                        lumpsize_dist[0,i] += 1
        # non-shear region                
            labeled_array, num_feature = labeled_obj(cldmask_ns,1,1)
            for label in np.unique(labeled_array)[1:]:
                tmp = len(np.where(labeled_array == label)[0]) # pixel numbers: size
                for i in range(len(size_cri)-1):
                    if (tmp >= size_cri[i]) and (tmp < size_cri[i+1]):
                        lumpsize_dist[1,i] += 1
                         
        except:
            continue
            
    return lumpsize_dist

In [ ]:
%%time
size_cri = np.linspace(0,400,101)
psize_dist = np.zeros((len(exp_label),2,len(size_cri)-1))
for n,prate in enumerate([prate1,prate2,prate3,prate4,prate5,prate6]):
    tmp = prate[-360:,:,:]
    psize_dist[n,:,:] = lumpsize_dist(tmp)

In [ ]:
nrow = [0,0,1,1,2,2,3]
ncol = [0,1,0,1,0,1,0]
fig,ax = plt.subplots(3,2,figsize=(10,15))

for n in range(len(exp_label)):
    ax[nrow[n],ncol[n]].plot(4*size_cri[:-1], psize_dist[n,0,:]/np.sum(psize_dist[n,0,:]),label='Shear region')
    ax[nrow[n],ncol[n]].plot(4*size_cri[:-1], psize_dist[n,1,:]/np.sum(psize_dist[n,1,:]),label='Non-Shear region')
    if n > 0:
        ref_dist = (psize_dist[0,1,:]/np.sum(psize_dist[0,1,:]) + psize_dist[0,1,:]/np.sum(psize_dist[0,1,:]))/2
        ax[nrow[n],ncol[n]].plot(4*size_cri[:-1], ref_dist ,color='lightgrey')
    ax[nrow[n],ncol[n]].set_yscale('log')
    ax[nrow[n],ncol[n]].legend(frameon=False,fontsize=11)
    ax[nrow[n],ncol[n]].set_xlabel('Size [km$^2$]',fontsize=13)
    ax[nrow[n],ncol[n]].set_ylabel('probability',fontsize=13)
    ax[nrow[n],ncol[n]].text(100,1e-1,exp_label[n],fontsize=13)
    ax[nrow[n],ncol[n]].set_ylim([1e-5,0])
    ax[nrow[n],ncol[n]].grid(linestyle=':')
    
    if n == 7:
        ax[nrow[n],ncol[n]].spines['bottom'].set_edegcolor('white')
        ax[nrow[n],ncol[n]].spines['top'].set_edgecolor('w') 
        ax[nrow[n],ncol[n]].spines['right'].set_edgecolor('w')
        ax[nrow[n],ncol[n]].spines['left'].set_edgecolor('w')
        #ax[nrow[n],ncol[n]].set_xticks([])
        #ax[nrow[n],ncol[n]].set_yticks([])
        
fig.savefig(fig_out+'objsize_PDF.png',dpi=200,bbox_inches='tight')

### Organization gradient (OG)

In [ ]:
Iorg_time = np.load('/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/Iorg_com_shear.npy')
SCAI_time = np.load('/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/SCAI_com_shear.npy')
COP_time = np.load('/data2/willytsai/cm1r19.8/analysis/runs_cheyenne/COP_com_shear.npy')

In [ ]:
color_label=['k','darkred','r','darkgreen','limegreen','orange','gold']

shear_int = [0,7.5,6,5,3.5,2.5]
shear_depth = [0,2,2,2,2,2,2]
fig = plt.figure(figsize=(5,5))
for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):
    
    plt.plot(s_int,SCAI_time[n,0,-360:].mean(),marker='^',markersize=12,linestyle='None'
                 ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # SCAI_BT
    plt.plot(s_int,SCAI_time[n,1,-360:].mean(),marker='x',markersize=12,linestyle='None'
                 ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # SCAI_BT
    
plt.xlabel('shear intensity',fontsize=15)
plt.ylabel('SCAI',fontsize=15);#plt.ylim([-6,0])
plt.legend(exp_label,fontsize=9)
plt.grid(linestyle=":")
plt.tick_params(labelsize=13)
plt.title('SCAI',fontsize=14)

# for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):

#     plt.plot(s_int,SCAI_time[n,0,-360:].mean()-SCAI_time[n,1,-360:].mean(),marker='o',markersize=12,linestyle='None'
#                  ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # SCAI_BT

In [ ]:
#shear_int = [0,3.75,7.5,2.5,5,1.25,2.5]
#shear_depth = [0,4,2,4,2,4,2]
fig,ax = plt.subplots(1,2,figsize=(10,5))

for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):

    ax[0].plot(s_int,COP_time[n,0,-360:].mean(),marker='^',markersize=13,linestyle='None'
                 ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # COP
ax[0].legend(exp_label,fontsize=10)

for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):
    ax[0].plot(s_int,COP_time[n,1,-360:].mean(),marker='x',markersize=13,linestyle='None'
                 ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # COP

for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):

    ax[1].plot(s_int,COP_time[n,0,-360:].mean()-COP_time[n,1,-360:].mean(),marker='o',markersize=12,linestyle='None'
                 ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # COP
ax[1].legend(exp_label,fontsize=10)
    
ax[0].set_xlabel('shear intensity [s$^{-1}$]',fontsize=15)
ax[1].set_xlabel('shear intensity [s$^{-1}$]',fontsize=15)
ax[0].set_ylabel('COP',fontsize=18);#plt.ylim([0,10])
ax[1].set_ylabel('$\Delta$COP',fontsize=18);#plt.ylim([0,10])
ax[0].grid(linestyle=":")
ax[1].grid(linestyle=":")
ax[0].tick_params(labelsize=13)
ax[1].tick_params(labelsize=13)
ax[1].set_title('COP difference (S-NS)')
plt.tight_layout()

fig.savefig(fig_out+'COP_shearintensity.png',dpi=200,bbox_inches='tight')
# for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):

#      plt.plot(s_int,COP_pr_time[n,0,-360:].mean()-COP_pr_time[n,1,-360:].mean(),marker='o',markersize=12,linestyle='None'
#                  ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # COP_pr

In [ ]:
pwat5

In [ ]:
## standardizing, centered PW 
fig = plt.figure(figsize=(13,4))
plt.contourf(t_dim,y_dim,pwat5.mean(axis=2).T,cmap='')
plt.colorbar()

In [ ]:
pwat_standardized = (pwat5-pwat5.mean(axis=(1,2)))/pwat5.std(axis=(1,2))

In [ ]:
def centered_pwat_xcolumn(pwat_stded_xcolumn):
    L = len(pwat_stded_xcolumn)
    idx = np.where(pwat_stded_xcolumn.values == np.max(pwat_stded_xcolumn).values)[0]
    pwat_re_xcolumn = np.roll(pwat_stded_xcolumn,int(L/2)-idx)
    pwat_mean_xcolumn = pwat_stded_xcolumn.mean().values
    
    return pwat_re_xcolumn, pwat_mean_xcolumn  

In [ ]:
idx = np.where(pwat_standardized[400,:,65].values == np.max(pwat_standardized[400,:,65]).values)[0]
test = np.roll(pwat_standardized[400,:,65],int(len(x_dim)/2)-idx)

In [ ]:
plt.plot(pwat_standardized[400,:,65],'k')
plt.plot(test)

In [ ]:
pwat_centered_pattern = np.empty(pwat_standardized[0,:,:].shape)
pwat_sorted_pattern = np.copy(pwat_standardized)
pwat_mean_xcolumn = np.empty(len(x_dim))

for t in range(len(t_dim)):
    for i in range(216):
        # 2D centered pattern
        pwat_centered_pattern[:,i], pwat_mean_xcolumn[i] = centered_pwat_xcolumn(pwat_standardized[t,:,i])
    
    idx_sorted = np.argsort(pwat_mean_xcolumn)
    pwat_sorted_pattern[t,:,:] = pwat_centered_pattern[:,idx_sorted]

In [ ]:
plt.pcolor(pwat_sorted_pattern[600,:,:],cmap='RdBu_r',vmin=-2,vmax=2)
plt.colorbar()

In [ ]:
plt.pcolor(pwat_sorted_pattern[-360:,:,162].T,vmax=3,vmin=-0.5)
plt.colorbar()
plt.contour(pwat_sorted_pattern[-360:,:,162].T,levels=[1])

### Cold pool intensity calculation
cold pool = vint_(theta_purt/theta_env)dz

In [ ]:
tmp = th1_3d[500,:,:,:]
buoy = 9.8*(tmp-tmp.mean(('lat','lon')))/tmp.mean(('lat','lon')) # g*th'/th_env
idx = np.where(buoy[:,150,200]<-0.005)[0]
h=0
for n in range(len(idx)-1):
    if (idx[n+1] - idx[n]) <=2: # allowing one discontinuity
        h = idx[n+1]
    else:
        break

In [ ]:
def coldpool_intensity(t):
    
    for i in range(len(y_dim)):
        for j in range(len(x_dim)):
            idx = np.where(buoy[t,:,i,j]<-0.005)[0]  # H = highest altitude of −0.005 m s−2
            h=0
            for n in range(len(idx)-1):
                if (idx[n+1] - idx[n]) <=2: # allowing one discontinuity
                    h = idx[n+1]
                else:
                    break
            if h > 0:        
                cp_den[i,j] = np.sqrt(2*np.trapz(-buoy[t,:h+1,i,j],z_dim[:h+1]*1000)) # cold pool speed[m/s]
            
    return cp_den

In [ ]:

buoy = 9.8*(th2_3d-th2_3d.mean(('lat','lon')))/th2_3d.mean(('lat','lon')) # g*th'/th_env
cp_den = np.zeros((len(t_dim),len(y_dim),len(x_dim)))*np.nan

for t in range(2):
    for i in range(len(y_dim)):
        for j in range(len(x_dim)):
            idx = np.where(buoy[t,:,i,j]<-0.005)[0]  # H = highest altitude of −0.005 m s−2
            h=0
            for n in range(len(idx)-1):
                if (idx[n+1] - idx[n]) <=2: # allowing one discontinuity
                    h = idx[n+1]
                else:
                    break
            if h > 0:        
                cp_den[t,i,j] = np.sqrt(2*np.trapz(-buoy[t,:h+1,i,j],z_dim[:h+1]*1000)) # cold pool speed[m/s]

In [ ]:
fig = plt.figure(figsize=(8,7))

plt.pcolor(x_dim,y_dim,cp_den[500,:,:],cmap='GnBu',vmin=0,vmax=15)
cbar = plt.colorbar(shrink=0.7)
cbar.set_label('Cold pool intendsity [m/s]',fontsize=13)
plt.contour(x_dim,y_dim,prate2[500,:,:],levels=[10,20],colors=['m'])

In [ ]:
# shear_int = [0,3.75,7.5,2.5,5,1.25,2.5]
# shear_depth = [0,4,2,4,2,4,2]
# fig = plt.figure(figsize=(6,6))
# for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):

#      plt.plot(s_int,Iorg_time[n,0,-360:].mean(),marker='^',markersize=12,linestyle='None'
#                  ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # Iorg

# plt.xlabel('shear intensity',fontsize=15)
# plt.ylabel('Iorg',fontsize=15);#plt.ylim([0,10])
# plt.legend(exp_label,loc=1,fontsize=9)
# plt.grid(linestyle=":")
# plt.tick_params(labelsize=13)
# plt.title('Iorg in the shear region')

# for n,(s_int,s_dep) in enumerate(zip(shear_int,shear_depth)):

#      plt.plot(s_int,Iorg_pr_time[n,0,-360:].mean(),marker='o',markersize=12,linestyle='None'
#                  ,color=color_label[n],markerfacecolor='None',markeredgewidth=2) # Iorg_pr

In [ ]:
# vertical dynamic structures of soundings over 6 places
y_pos = [26,52,78,104,130,156,182]
color_snds = ['green','r','purple','purple','purple','r','green']
nrow = [0,0,1,1,2,2,3]
ncol = [0,1,0,1,0,1,0]
pos_label = ['NS','EG1','S','S','S','EG2','NS']

fig,ax = plt.subplots(4,2,figsize=(8,12))

for n,(w,ro) in enumerate(zip([w1,w2,w3,w4,w5,w6,w7],[ro1,ro2,ro3,ro4,ro5,ro6,ro7])):
    
    for p,y in enumerate(y_pos):
        ax[nrow[n],ncol[n]].plot(w[-360:,:,y,0].mean(axis=0),z_dim,color=color_snds[p])
        ax[nrow[n],ncol[n]].set_ylabel('Height [km]')
        ax[nrow[n],ncol[n]].set_xlabel('Vert. velocity [m/s]')
        ax[nrow[n],ncol[n]].text(0.02,16,exp_label[n],fontsize=12)
        ax[nrow[n],ncol[n]].set_ylim([0,18])
        ax[nrow[n],ncol[n]].vlines(x=0,ymin=0,ymax=18,color='lightgrey')
        ax[nrow[n],ncol[n]].set_xlim([-0.03,0.05])

plt.tight_layout()

In [ ]:
# prec vs CWV diagram 
pwat_cri = np.linspace(50,80,31)
prate_s_com = np.zeros((len(exp_label),len(pwat_cri)-1))
prate_ns_com = np.copy(prate_s_com)

fig,ax = plt.subplots(4,2,figsize=(10,15))

nrow = [0,0,1,1,2,2,3]
ncol = [0,1,0,1,0,1,0]

for n,(prate,pwat) in enumerate(zip([prate1,prate2,prate3,prate4,prate5,prate6]
                                  ,[pwat1,pwat2,pwat3,pwat4,pwat5,pwat6])):
    #S
    prate_s = prate[-360:,65:151,:].values.flatten()
    pwat_s = pwat[-360:,65:151,:].values.flatten()
    #NS
    prate_ns = np.concatenate((prate[-360:,:43,:],prate[-360:,-43:,:])).flatten()
    pwat_ns = np.concatenate((pwat[-360:,:43,:],pwat[-360:,-43:,:])).flatten()
    
    prate_s_bin = np.zeros(len(pwat_cri)-1)
    prate_ns_bin = np.zeros(len(pwat_cri)-1)
    for i in range(len(pwat_cri)-1):
        idx_s = np.where(np.logical_and(pwat_s>=pwat_cri[i],pwat_s<pwat_cri[i+1]))[0]
        idx_ns = np.where(np.logical_and(pwat_ns>=pwat_cri[i],pwat_ns<pwat_cri[i+1]))[0]
        prate_s_bin[i] = np.nanmean(prate_s[idx_s])
        prate_ns_bin[i] = np.nanmean(prate_ns[idx_ns])
        
    ax[nrow[n],ncol[n]].plot(pwat_cri[:-1],prate_s_bin/24,label='Shear')
    ax[nrow[n],ncol[n]].plot(pwat_cri[:-1],prate_ns_bin/24,label='Non-Shear')
    ax[nrow[n],ncol[n]].legend()
    ax[nrow[n],ncol[n]].text(56.5,20,exp_label[n],fontsize=13)
    ax[nrow[n],ncol[n]].set_ylabel('P [mm/hr]',fontsize=13)
    ax[nrow[n],ncol[n]].grid(linestyle=':')
    if nrow[n] == 3:
        ax[nrow[n],ncol[n]].set_xlabel('CWV [mm]',fontsize=13)
    ax[nrow[n],ncol[n]].set_xlim([55,80])
    ax[nrow[n],ncol[n]].set_ylim([-2,30])
    
    # save data
    prate_s_com[n,:] = prate_s_bin/24
    prate_ns_com[n,:] = prate_ns_bin/24

In [ ]:
from scipy.stats import linregress

In [ ]:
# p-cwv relationship: prate = exp(a*pwat + b), fixing b and see how a depends on "org" 
slope_com = np.empty(len(exp_label))
interp_com = np.copy(slope_com) 

for n in range(len(exp_label)):
    idx_rmnan = np.where(np.logical_and(np.isnan(prate_s_com[n,:])==0,prate_s_com[n,:]>0))[0]
    slope,interp,r,p_val,stderr = linregress(pwat_cri[idx_rmnan[:]],np.log(prate_s_com[n,idx_rmnan]))
    slope_com[n] = slope
    interp_com[n] = interp

# recalculating modified parameter "a", with fixed b for every case
slope_range = np.linspace(0,0.5,50)
slope_mod = np.empty(len(exp_label))

# for n in range(len(exp_label)):
    
#     diff_p = np.empty(len(slope_range))
#     for i,slp in enumerate(slope_range):
#         p_new = np.exp(slp*pwat_cri[:-1] + interp_com[0])
#         diff_p[i] = np.abs(np.sum(p_new-prate_s_com[n,:]))
#     idx_min = np.where(diff_p==np.min(diff_p))[0]
#     slope_mod[n] = slope_range[idx_min]

In [ ]:
#for n in range(7):
fig, ax = plt.subplots(1,2,figsize=(12,5))

for n in [0,2,4,6]:
    ax[0].plot(pwat_cri[:-1],prate_s_com[n,:]-prate_s_com[0,:],color=color_label[n]
             ,label=exp_label[n],linewidth=2)
    ax[1].plot(pwat_cri[:-1],prate_ns_com[n,:]-prate_ns_com[0,:],color=color_label[n]
             ,label=exp_label[n],linewidth=2)
    
ax[0].legend()
ax[0].set_xlim([55,79])
ax[0].set_xlabel('CWV [mm]',fontsize=13)
ax[0].set_ylabel('P [mm/hr]',fontsize=13)
ax[0].grid(linestyle=':')
ax[0].set_title('Shear region',fontsize=13)

#ax[1].legend()
ax[1].set_xlim([55,79])
ax[1].set_xlabel('CWV [mm]',fontsize=13)
ax[1].set_ylabel('P [mm/hr]',fontsize=13)
ax[1].grid(linestyle=':')
ax[1].set_title('Non-shear region',fontsize=13)

In [ ]:
for n in range(len(exp_label)):
    plt.plot(n,slope_com[n],'o',color=color_label[n],label=exp_label[n])
plt.legend(ncol=2)

In [ ]:
Z = np.tile(z_dim,(len(y_dim),1)).swapaxes(0,1)
Z.shape

In [ ]:
# MSE budget analysis over the meridional direction

sfx = 372.728*np.ones(len(y_dim)) # [W/m^2]
vint_QR = vint_QR*np.ones(len(y_dim)) # [W/m^2]

dvint_h_dt = np.empty((len(exp_label),len(y_dim)))
vint_vadv_h = np.copy(dvint_h_dt)
vint_hadv_h = np.copy(dvint_h_dt)

for n,(T,qv,ro,w,v) in enumerate(zip([T1,T2,T3,T4,T5,T6],[qv1,qv2,qv3,qv4,qv5,qv6],
                                 [ro1,ro2,ro3,ro4,ro5,ro6],[w1,w2,w3,w4,w5,w6],[v1,v2,v3,v4,v5,v6])):
    
    h = 1004*(T[-360:,:,:,0]+273.15)+2.5e6*qv[-360:,:,:,0] \
            +9.8*np.tile(z_dim*1000,reps=(360,len(y_dim),1)).swapaxes(1,2)
    
    dhdz = np.gradient(h,z_dim*1000,axis=1)
    dhdy = np.gradient(h,y_dim*2000,axis=2)
    
    vadv_h = np.trapz(ro[-360:,:,:,0]*w[-360:,:,:,0]*dhdz,z_dim.values*1000,axis=1) # vertically integrated vadv
    hadv_h = np.trapz(ro[-360:,:,:,0]*v[-360:,:,:,0]*dhdy,z_dim.values*1000,axis=1) # vertically integrated hadv
    vint_h = np.trapz(ro[-360:,:,:,0]*h,z_dim.values*1000,axis=1) # vertically integrated MSE   
    
    dvint_h_dt[n,:] = np.gradient(vint_h,3600,axis=0).mean(axis=0) # [w/m^2]
    vint_vadv_h[n,:] = vadv_h.mean(axis=0)
    vint_hadv_h[n,:] = hadv_h.mean(axis=0)

In [ ]:
fig,ax = plt.subplots(4,2,figsize=(10,15))

nrow = [0,0,1,1,2,2,3]
ncol = [0,1,0,1,0,1,0]

for n in range(len(exp_label)):
    ax[nrow[n],ncol[n]].plot(dvint_h_dt[n,:],'--k')
#    ax[nrow[n],ncol[n]].plot(sfx+vint_QR,color='k')
    ax[nrow[n],ncol[n]].plot(-vint_hadv_h[n,:],color='b')
#    ax[nrow[n],ncol[n]].plot(-vint_vadv_h[n,:],color='r')

In [ ]:
fig = plt.figure(figsize=(8,6))
for n in range(6):
    plt.plot(y_dim,running_mean(-vint_vadv_h[n,:],5),'-',color=color_label[n])
    #plt.plot(y_dim,-vint_hadv_h[n,:],color=color_label[n])
    #plt.plot(y_dim,vint_QR,'grey')
    #plt.plot(y_dim,sfx,'k')
    #plt.legend(['VADV_MSE','HADV_MSE','QR','SFX'],frameon=False)

In [ ]:
plt.plot(y_dim,-vint_vadv_h[4,:],'r')
plt.plot(y_dim,-vint_hadv_h[4,:],'b')
plt.plot(y_dim,vint_QR,'grey')
plt.plot(y_dim,sfx,'k')
plt.legend(['VADV_MSE','HADV_MSE','QR','SFX'],frameon=False)

In [ ]:
# np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/Iorg_com_shear.npy',Iorg_time)
# np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/MICA_com_shear.npy',MICA_time)
# np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/SCAI_com_shear.npy',SCAI_time)
# np.save('/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne/COP_com_shear.npy',COP_time)